<a href="https://colab.research.google.com/github/cesphamm/procesamiento_lenguaje_natural/blob/main/Desafio_4_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## LSTM Traductor (PyTorch)
Ejemplo basado en [LINK](https://stackabuse.com/python-for-nlp-neural-machine-translation-with-seq2seq-in-keras/)


### 1 - Datos
El objecto es utilizar datos disponibles del Tatoeba Project de traducciones de texto en diferentes idiomas. Se construirá un modelo traductor de inglés a español seq2seq utilizando encoder-decoder.\
[LINK](https://www.manythings.org/anki/)


In [1]:
#!pip install torch torchvision


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import time

print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(f"Dispositivo: {device}")


CUDA disponible: False
Dispositivo: mps


In [3]:
import re

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import os
from pathlib import Path
from io import StringIO
import pickle


/Users/cespinola/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
# Descargar la carpeta de dataset

import os
if os.access('spa-eng', os.F_OK) is False:
    if os.access('spa-eng.zip', os.F_OK) is False:
        !curl -L -o 'spa-eng.zip' 'http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip'
    !unzip -q spa-eng.zip
else:
    print("El dataset ya se encuentra descargado")


El dataset ya se encuentra descargado


In [5]:
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True  # Optimiza kernels para tu hardware
    torch.backends.cudnn.deterministic = False  # Permite optimizaciones no deterministas

In [6]:
# dataset_file

text_file = "./spa-eng/spa.txt"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]

# Ahora podemos usar más datos gracias al DataLoader
# que carga los batches on-the-fly sin consumir toda la RAM
MAX_NUM_SENTENCES = 118964 #50000

# Mezclar el dataset, forzar semilla siempre igual
np.random.seed([40])
np.random.shuffle(lines)

input_sentences = []
output_sentences = []
output_sentences_inputs = []
count = 0

for line in lines:
    count += 1
    if count > MAX_NUM_SENTENCES:
        break

    # el tabulador señaliza la separación entre las oraciones
    # en ambos idiomas
    if '\t' not in line:
        continue

    # Input sentence --> eng
    # output --> spa
    input_sentence, output = line.rstrip().split('\t')

    # output sentence (decoder_output) tiene <eos>
    output_sentence = output + ' <eos>'
    # output sentence input (decoder_input) tiene <sos>
    output_sentence_input = '<sos> ' + output

    input_sentences.append(input_sentence)
    output_sentences.append(output_sentence)
    output_sentences_inputs.append(output_sentence_input)

print("Cantidad de rows disponibles:", len(lines))
print("Cantidad de rows utilizadas:", len(input_sentences))


Cantidad de rows disponibles: 118964
Cantidad de rows utilizadas: 118964


In [7]:
input_sentences[0], output_sentences[0], output_sentences_inputs[0]


('A deal is a deal.',
 'Un trato es un trato. <eos>',
 '<sos> Un trato es un trato.')

### 2 - Preprocesamiento


In [8]:
# Definir el tamaño máximo del vocabulario
MAX_VOCAB_SIZE = 25000
# Vamos a necesitar un tokenizador para cada idioma


In [9]:
# Tokenizar las palabras con el Tokenizer de Keras
# Definir una máxima cantidad de palabras a utilizar:
# - num_words --> el número máximo de palabras a conservar, en función de la frecuencia de las palabras.
# - Solo se conservarán las num_words-1 palabras más comunes.

# tokenizador de inglés
input_tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
input_tokenizer.fit_on_texts(input_sentences)
input_integer_seq = input_tokenizer.texts_to_sequences(input_sentences)

word2idx_inputs = input_tokenizer.word_index
print("Palabras en el vocabulario:", len(word2idx_inputs))

max_input_len = max(len(sen) for sen in input_integer_seq)
print("Sentencia de entrada más larga:", max_input_len)


Palabras en el vocabulario: 13524
Sentencia de entrada más larga: 47


In [10]:
# tokenizador de español
# A los filtros de símbolos del Tokenizer agregamos el "¿",
# sacamos los "<>" para que no afectar nuestros tokens
output_tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, filters='!"#$%&()*+,-./:;=¿?@[\\]^_`{|}~\t\n')
output_tokenizer.fit_on_texts(["<sos>", "<eos>"] + output_sentences)
output_integer_seq = output_tokenizer.texts_to_sequences(output_sentences)
output_input_integer_seq = output_tokenizer.texts_to_sequences(output_sentences_inputs)

word2idx_outputs = output_tokenizer.word_index
print("Palabras en el vocabulario:", len(word2idx_outputs))

num_words_output = min(len(word2idx_outputs) + 1, MAX_VOCAB_SIZE)
# Se suma 1 para incluir el token de palabra desconocida

max_out_len = max(len(sen) for sen in output_integer_seq)
print("Sentencia de salida más larga:", max_out_len)


Palabras en el vocabulario: 26341
Sentencia de salida más larga: 50


Como era de esperarse, las sentencias en castellano son más largas que en inglés, y lo mismo sucede con su vocabulario.


In [11]:
# Por una cuestion de que no explote la RAM se limitará el tamaño de las sentencias de entrada
# a la mitad:
max_input_len = 16
max_out_len = 18


A la hora de realiza padding es importante tener en cuenta que en el encoder los ceros se agregan al comienzo y en el decoder al final. Esto es porque la salida del encoder está basado en las últimas palabras de la sentencia (son las más importantes), mientras que en el decoder está basado en el comienzo de la secuencia de salida ya que es la realimentación del sistema y termina con fin de sentencia.


In [12]:
print("Cantidad de rows del dataset:", len(input_integer_seq))

encoder_input_sequences = pad_sequences(input_integer_seq, maxlen=max_input_len)
print("encoder_input_sequences shape:", encoder_input_sequences.shape)

decoder_input_sequences = pad_sequences(output_input_integer_seq, maxlen=max_out_len, padding='post')
print("decoder_input_sequences shape:", decoder_input_sequences.shape)


Cantidad de rows del dataset: 118964
encoder_input_sequences shape: (118964, 16)
decoder_input_sequences shape: (118964, 18)


La última capa del modelo (softmax) necesita que los valores de salida
del decoder (decoder_sequences) estén en formato oneHotEncoder.\
Se utiliza "decoder_output_sequences" con la misma estrategia con que se transformó la entrada del decoder.


In [13]:
# Ya no creamos decoder_targets completo en memoria
# Solo guardamos las secuencias de índices, PyTorch CrossEntropyLoss trabaja con índices
decoder_output_sequences = pad_sequences(output_integer_seq, maxlen=max_out_len, padding='post')
print("decoder_output_sequences shape:", decoder_output_sequences.shape)


decoder_output_sequences shape: (118964, 18)


#### Dataset para optimizar el uso de RAM

El problema principal de memoria es que `decoder_targets` (el one-hot encoding de las salidas) consume muchísima RAM:
- Con 50,000 muestras × 18 timesteps × 6,000 palabras vocabulario × 4 bytes = **~21 GB de RAM**

El `Dataset` de PyTorch:
1. Hereda de `torch.utils.data.Dataset` para integrarse con PyTorch
2. PyTorch CrossEntropyLoss trabaja con índices directamente, sin necesidad de one-hot
3. El DataLoader genera los batches on-the-fly
4. Permite mezclar los datos (shuffle) usando DataLoader


In [14]:
class TranslationDataset(Dataset):
    """
    Dataset para el modelo de traducción seq2seq en PyTorch.
    Genera muestras on-the-fly para evitar cargar todo el dataset en RAM.
    """

    def __init__(self, encoder_input, decoder_input, decoder_output):
        """
        Args:
            encoder_input: Secuencias de entrada del encoder (ya con padding)
            decoder_input: Secuencias de entrada del decoder (ya con padding)
            decoder_output: Secuencias de salida del decoder (índices, NO one-hot)
        """
        self.encoder_input = torch.LongTensor(encoder_input)
        self.decoder_input = torch.LongTensor(decoder_input)
        self.decoder_output = torch.LongTensor(decoder_output)

    def __len__(self):
        return len(self.encoder_input)

    def __getitem__(self, idx):
        return (self.encoder_input[idx],
                self.decoder_input[idx],
                self.decoder_output[idx])


### 3 - Preparar los embeddings


In [15]:
#import sys
#!{sys.executable} -m pip install gdown


In [16]:
# Descargar los embeddings desde un google drive (es la forma más rápida)
# NOTA: No hay garantía de que estos links perduren, en caso de que no estén
# disponibles descargar de la página oficial como se explica en el siguiente bloque de código
import os
import gdown
if os.access('gloveembedding.pkl', os.F_OK) is False:
    url = 'https://drive.google.com/uc?id=1KY6avD5I1eI2dxQzMkR3WExwKwRq2g94&export=download'
    output = 'gloveembedding.pkl'
    gdown.download(url, output, quiet=False)
else:
    print("Los embeddings gloveembedding.pkl ya están descargados")


Los embeddings gloveembedding.pkl ya están descargados


In [17]:
# En caso de que gdown de algún error de permisos intentar descargar los
# embeddings con curl:

#!curl -L -o 'gloveembedding.pkl' 'https://drive.google.com/u/0/uc?id=1KY6avD5I1eI2dxQzMkR3WExwKwRq2g94&export=download&confirm=t'


In [18]:
class WordsEmbeddings(object):
    logger = logging.getLogger(__name__)

    def __init__(self):
        # load the embeddings
        words_embedding_pkl = Path(self.PKL_PATH)
        if not words_embedding_pkl.is_file():
            words_embedding_txt = Path(self.WORD_TO_VEC_MODEL_TXT_PATH)
            assert words_embedding_txt.is_file(), 'Words embedding not available'
            embeddings = self.convert_model_to_pickle()
        else:
            embeddings = self.load_model_from_pickle()
        self.embeddings = embeddings
        # build the vocabulary hashmap
        index = np.arange(self.embeddings.shape[0])
        # Dicctionarios para traducir de embedding a IDX de la palabra
        self.word2idx = dict(zip(self.embeddings['word'], index))
        self.idx2word = dict(zip(index, self.embeddings['word']))

    def get_words_embeddings(self, words):
        words_idxs = self.words2idxs(words)
        return self.embeddings[words_idxs]['embedding']

    def words2idxs(self, words):
        return np.array([self.word2idx.get(word, -1) for word in words])

    def idxs2words(self, idxs):
        return np.array([self.idx2word.get(idx, '-1') for idx in idxs])

    def load_model_from_pickle(self):
        self.logger.debug(
            'loading words embeddings from pickle {}'.format(
                self.PKL_PATH
            )
        )
        max_bytes = 2**28 - 1 # 256MB
        bytes_in = bytearray(0)
        input_size = os.path.getsize(self.PKL_PATH)
        with open(self.PKL_PATH, 'rb') as f_in:
            for _ in range(0, input_size, max_bytes):
                bytes_in += f_in.read(max_bytes)
        embeddings = pickle.loads(bytes_in)
        self.logger.debug('words embeddings loaded')
        return embeddings

    def convert_model_to_pickle(self):
        # create a numpy strctured array:
        # word     embedding
        # U50      np.float32[]
        # word_1   a, b, c
        # word_2   d, e, f
        # ...
        # word_n   g, h, i
        self.logger.debug(
            'converting and loading words embeddings from text file {}'.format(
                self.WORD_TO_VEC_MODEL_TXT_PATH
            )
        )
        structure = [('word', np.dtype('U' + str(self.WORD_MAX_SIZE))),
                     ('embedding', np.float32, (self.N_FEATURES,))]
        structure = np.dtype(structure)
        # load numpy array from disk using a generator
        with open(self.WORD_TO_VEC_MODEL_TXT_PATH, encoding="utf8") as words_embeddings_txt:
            embeddings_gen = (
                (line.split()[0], line.split()[1:]) for line in words_embeddings_txt
                if len(line.split()[1:]) == self.N_FEATURES
            )
            embeddings = np.fromiter(embeddings_gen, structure)
        # add a null embedding
        null_embedding = np.array(
            [('null_embedding', np.zeros((self.N_FEATURES,), dtype=np.float32))],
            dtype=structure
        )
        embeddings = np.concatenate([embeddings, null_embedding])
        # dump numpy array to disk using pickle
        max_bytes = 2**28 - 1 # # 256MB
        bytes_out = pickle.dumps(embeddings, protocol=pickle.HIGHEST_PROTOCOL)
        with open(self.PKL_PATH, 'wb') as f_out:
            for idx in range(0, len(bytes_out), max_bytes):
                f_out.write(bytes_out[idx:idx+max_bytes])
        self.logger.debug('words embeddings loaded')
        return embeddings


class GloveEmbeddings(WordsEmbeddings):
    WORD_TO_VEC_MODEL_TXT_PATH = 'glove.twitter.27B.50d.txt'
    PKL_PATH = 'gloveembedding.pkl'
    N_FEATURES = 50
    WORD_MAX_SIZE = 60

class FasttextEmbeddings(WordsEmbeddings):
    WORD_TO_VEC_MODEL_TXT_PATH = 'cc.en.300.vec'
    PKL_PATH = 'fasttext.pkl'
    N_FEATURES = 300
    WORD_MAX_SIZE = 60


In [19]:
# Por una cuestion de RAM se utilizarán los embeddings de Glove de dimension 50
model_embeddings = GloveEmbeddings()


In [20]:
# Crear la Embedding matrix de las secuencias
# en inglés

print('preparing embedding matrix...')
embed_dim = model_embeddings.N_FEATURES
words_not_found = []

# word_index provieen del tokenizer

nb_words = min(MAX_VOCAB_SIZE, len(word2idx_inputs)) # vocab_size
embedding_matrix = np.zeros((nb_words, embed_dim))

if embedding_matrix.shape[0] < (np.max(encoder_input_sequences) + 1):
    diff = (np.max(encoder_input_sequences) + 1) - embedding_matrix.shape[0]
    embedding_matrix = np.vstack([embedding_matrix,
                                  np.zeros((diff, embedding_matrix.shape[1]))])

for word, i in word2idx_inputs.items():
    if i >= nb_words:
        continue
    embedding_vector = model_embeddings.get_words_embeddings(word)[0]
    if (embedding_vector is not None) and len(embedding_vector) > 0:

        embedding_matrix[i] = embedding_vector
    else:
        # words not found in embedding index will be all-zeros.
        words_not_found.append(word)

print('number of null word embeddings:', np.sum(np.sum(embedding_matrix**2, axis=1) == 0))


preparing embedding matrix...
number of null word embeddings: 219


In [21]:
# Dimensión de los embeddings de la secuencia en inglés
embedding_matrix.shape


(13525, 50)

### 4 - Entrenar el modelo


#### Construir modelos


In [22]:
embed_dim = model_embeddings.N_FEATURES
max_input_len


16

In [23]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_matrix, n_units):
        super(Encoder, self).__init__()
        #vocab_size, embed_dim = embedding_matrix.shape
        #self.embedding = nn.Embedding(vocab_size, embed_dim)
        #self.embedding.weight = nn.Parameter(torch.FloatTensor(embedding_matrix))
        #self.embedding.weight.requires_grad = False  # No entrenable
        #self.lstm = nn.LSTM(embed_dim, n_units, batch_first=True)

        self.lstm_size = n_units
        self.num_layers = 1
        self.embedding_dim = embed_dim
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=self.embedding_dim, padding_idx=0)
        self.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))
        self.embedding.weight.requires_grad = False  # marcar como layer no entrenable (freeze)
        self.lstm = nn.LSTM(input_size=self.embedding_dim, hidden_size=self.lstm_size, batch_first=True,
                            num_layers=self.num_layers) # LSTM layer

    def forward(self, x):
        out = self.embedding(x)
        lstm_output, (ht, ct) = self.lstm(out)
        return (ht, ct)


class Decoder(nn.Module):
    def __init__(self, vocab_size, num_words_output, n_units):
        super(Decoder, self).__init__()
        # num_embeddings = vocab_size, definido por le Tokenizador
        # embedding_dim = 50 --> dimensión de los embeddings utilizados
        self.lstm_size = n_units
        self.num_layers = 1
        self.embedding_dim = embed_dim
        self.output_dim = num_words_output

        #self.embedding = nn.Embedding(num_words_output, n_units)
        #self.lstm = nn.LSTM(n_units, n_units, batch_first=True)
        #self.fc = nn.Linear(n_units, num_words_output)
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=self.embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=self.embedding_dim, hidden_size=self.lstm_size, batch_first=True,
                            num_layers=self.num_layers) # LSTM layer
        self.fc1 = nn.Linear(in_features=self.lstm_size, out_features=self.output_dim) # Fully connected layer

        self.softmax = nn.Softmax(dim=1) # normalize in dim 1

    #def forward(self, x, h, c):
    #    x = self.embedding(x)
    #    outputs, (h, c) = self.lstm(x, (h, c))
    #    predictions = self.fc(outputs)
    #    return predictions, h, c

    def forward(self, x, prev_state):
        out = self.embedding(x)
        lstm_output, (ht, ct) = self.lstm(out, prev_state)
        out = self.softmax(self.fc1(lstm_output[:,-1,:])) # take last output (last seq)
        return out, (ht, ct)


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder

        assert encoder.lstm_size == decoder.lstm_size, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.num_layers == decoder.num_layers, \
            "Encoder and decoder must have equal number of layers!"

    def forward(self, encoder_input, decoder_input):
        batch_size = decoder_input.shape[0]
        decoder_input_len = decoder_input.shape[1]
        vocab_size = self.decoder.output_dim

        # tensor para almacenar la salida
        # (batch_size, sentence_len, one_hot_size)
        outputs = torch.zeros(
            batch_size, decoder_input_len, vocab_size, device=decoder_input.device)

        # ultimo hidden state del encoder, primer estado oculto del decoder
        prev_state = self.encoder(encoder_input)

        # En la primera iteracion se toma el primer token de target ()
        input = decoder_input[:, 0:1]

        for t in range(decoder_input_len):
            # t --> token index

            # utilizamos método "teacher forcing", es decir que durante
            # el entrenamiento no realimentamos la salida del decoder
            # sino el token correcto que sigue en target
            input = decoder_input[:, t:t+1]

            # ingresar cada token embedding, uno por uno junto al hidden state
            # recibir el output del decoder (softmax)
            output, prev_state = self.decoder(input, prev_state)
            top1 = output.argmax(1).view(-1, 1)

            # Sino se usará "teacher forcing" habría que descomentar
            # esta linea.
            # Hay ejemplos dandos vuelta en donde se utilza un random
            # para ver en cada vuelta que técnica se aplica
            #input = top1

            # guardar cada salida (softmax)
            outputs[:, t, :] = output
        return outputs


In [24]:
def build_traductor_model(n_units=128, encoder_input_sequences_m=None, embedding_matrix_m=None):
    """Construye un modelo de traducción seq2seq."""

    encoder_input_sequences_m = np.clip(
        encoder_input_sequences_m,
        0,
        embedding_matrix_m.shape[0] - 1
    )

    vocab_size = embedding_matrix_m.shape[0]

    encoder = Encoder(vocab_size=vocab_size, embedding_matrix=embedding_matrix_m, n_units=n_units)
    decoder = Decoder(vocab_size=vocab_size, num_words_output=num_words_output, n_units=n_units)
    model = Seq2Seq(encoder=encoder, decoder=decoder)
    model = model.to(device)
    if hasattr(torch, 'compile'):
        model = torch.compile(model, mode='reduce-overhead')

    model_env = {
        'model': model,
        'encoder': encoder,
        'decoder': decoder,
        'encoder_input_sequences': encoder_input_sequences_m,
        'n_units': n_units
    }
    return model_env


In [25]:
n_units_list = [64, 128, 256]
#n_units_list = [128, 256]
models = {f'n_units_{n_units}': None for n_units in n_units_list}

for n_units in n_units_list:
    model_env = build_traductor_model(n_units=n_units, encoder_input_sequences_m=encoder_input_sequences, embedding_matrix_m=embedding_matrix)
    models[f'n_units_{n_units}'] = model_env
    del model_env


#### Summary


In [26]:
m_64 = models['n_units_64']
m_128 = models['n_units_128']
m_256 = models['n_units_256']


In [27]:
print("Resumen del modelo con 64 unidades LSTM:")
print(m_64['model'])


Resumen del modelo con 64 unidades LSTM:
OptimizedModule(
  (_orig_mod): Seq2Seq(
    (encoder): Encoder(
      (embedding): Embedding(13525, 50, padding_idx=0)
      (lstm): LSTM(50, 64, batch_first=True)
    )
    (decoder): Decoder(
      (embedding): Embedding(13525, 50, padding_idx=0)
      (lstm): LSTM(50, 64, batch_first=True)
      (fc1): Linear(in_features=64, out_features=25000, bias=True)
      (softmax): Softmax(dim=1)
    )
  )
)


In [28]:
print(f"Modelo con 128 unidades:")
print(m_128['model'])


Modelo con 128 unidades:
OptimizedModule(
  (_orig_mod): Seq2Seq(
    (encoder): Encoder(
      (embedding): Embedding(13525, 50, padding_idx=0)
      (lstm): LSTM(50, 128, batch_first=True)
    )
    (decoder): Decoder(
      (embedding): Embedding(13525, 50, padding_idx=0)
      (lstm): LSTM(50, 128, batch_first=True)
      (fc1): Linear(in_features=128, out_features=25000, bias=True)
      (softmax): Softmax(dim=1)
    )
  )
)


In [29]:
print(f"Modelo con 256 unidades:")
print(m_256['model'])


Modelo con 256 unidades:
OptimizedModule(
  (_orig_mod): Seq2Seq(
    (encoder): Encoder(
      (embedding): Embedding(13525, 50, padding_idx=0)
      (lstm): LSTM(50, 256, batch_first=True)
    )
    (decoder): Decoder(
      (embedding): Embedding(13525, 50, padding_idx=0)
      (lstm): LSTM(50, 256, batch_first=True)
      (fc1): Linear(in_features=256, out_features=25000, bias=True)
      (softmax): Softmax(dim=1)
    )
  )
)


#### Visualizar arquitectura


In [30]:
print("Arquitectura del encoder 64:")
print(m_64['encoder'])


Arquitectura del encoder 64:
Encoder(
  (embedding): Embedding(13525, 50, padding_idx=0)
  (lstm): LSTM(50, 64, batch_first=True)
)


In [31]:
print("Arquitectura del decoder 64:")
print(m_64['decoder'])


Arquitectura del decoder 64:
Decoder(
  (embedding): Embedding(13525, 50, padding_idx=0)
  (lstm): LSTM(50, 64, batch_first=True)
  (fc1): Linear(in_features=64, out_features=25000, bias=True)
  (softmax): Softmax(dim=1)
)


#### Entrenar modelos


In [32]:
def train_model(model_env, train_loader, val_loader, epochs=50, patience=3):
    """Entrena el modelo con early stopping.
    Añade mediciones de tiempo por época y por step (train y val).
    """
    model = model_env['model']
    criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignorar padding
    optimizer = optim.Adam(model.parameters())

    # Scheduler para ajustar learning rate
    #scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=patience)

    history = {
        'loss': [], 'accuracy': [], 'val_loss': [], 'val_accuracy': [],
        'epoch_time': [], 'avg_train_step_time': [], 'avg_val_step_time': []
    }
    best_val_loss = float('inf')
    patience_counter = 0
    best_state = None

    print(f"Entrenando modelo con {epochs} épocas y paciencia {patience}")
    print(f"Device: {device}")

    for epoch in range(epochs):
        # --- Training ---
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        train_step_time_total = 0.0
        train_step_count = 0

        epoch_start = time.perf_counter()

        for encoder_input, decoder_input, decoder_output in train_loader:
            step_start = time.perf_counter()

            encoder_input = encoder_input.to(device, non_blocking=True)
            decoder_input = decoder_input.to(device, non_blocking=True)
            decoder_output = decoder_output.to(device, non_blocking=True)

            optimizer.zero_grad()
            outputs = model(encoder_input, decoder_input)

            # Reshape para calcular loss
            outputs_flat = outputs.view(-1, outputs.shape[-1])
            targets_flat = decoder_output.view(-1)

            loss = criterion(outputs_flat, targets_flat)
            loss.backward()

            # Gradient clipping para estabilidad
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()

            # Si usamos GPU, sincronizar antes de medir tiempo del step
            if device.type == 'cuda':
                torch.cuda.synchronize()
            step_end = time.perf_counter()

            train_step_time_total += (step_end - step_start)
            train_step_count += 1

            train_loss += loss.item()

            # Calcular accuracy
            with torch.no_grad():
                _, predicted = outputs.max(dim=-1)
                mask = decoder_output != 0
                train_correct += (predicted[mask] == decoder_output[mask]).sum().item()
                train_total += mask.sum().item()

        epoch_end_after_train = time.perf_counter()

        train_loss /= len(train_loader)
        train_acc = train_correct / train_total if train_total > 0 else 0
        avg_train_step_time = (train_step_time_total / train_step_count) if train_step_count > 0 else 0.0

        # --- Validation ---
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        val_step_time_total = 0.0
        val_step_count = 0

        with torch.no_grad():
            for encoder_input, decoder_input, decoder_output in val_loader:
                val_step_start = time.perf_counter()

                encoder_input = encoder_input.to(device, non_blocking=True)
                decoder_input = decoder_input.to(device, non_blocking=True)
                decoder_output = decoder_output.to(device, non_blocking=True)

                outputs = model(encoder_input, decoder_input)

                outputs_flat = outputs.view(-1, outputs.shape[-1])
                targets_flat = decoder_output.view(-1)

                loss = criterion(outputs_flat, targets_flat)
                val_loss += loss.item()

                _, predicted = outputs.max(dim=-1)
                mask = decoder_output != 0
                val_correct += (predicted[mask] == decoder_output[mask]).sum().item()
                val_total += mask.sum().item()

                # Sincronizar GPU si aplica antes de medir tiempo del step
                if device.type == 'cuda':
                    torch.cuda.synchronize()
                val_step_end = time.perf_counter()

                val_step_time_total += (val_step_end - val_step_start)
                val_step_count += 1

        epoch_end = time.perf_counter()

        val_loss /= len(val_loader)
        val_acc = val_correct / val_total if val_total > 0 else 0
        avg_val_step_time = (val_step_time_total / val_step_count) if val_step_count > 0 else 0.0

        epoch_time = epoch_end - epoch_start
        # Tiempo sólo de training dentro de la época
        train_only_time = epoch_end_after_train - epoch_start

        history['loss'].append(train_loss)
        history['accuracy'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_accuracy'].append(val_acc)
        history['epoch_time'].append(epoch_time)
        history['avg_train_step_time'].append(avg_train_step_time)
        history['avg_val_step_time'].append(avg_val_step_time)

        print(
            f'Epoch {epoch+1}/{epochs} - loss: {train_loss:.4f} - accuracy: {train_acc:.4f} - '
            f'val_loss: {val_loss:.4f} - val_accuracy: {val_acc:.4f}\n'
            f'    epoch_time: {epoch_time:.3f}s (train_time: {train_only_time:.3f}s) - '
            f'avg_train_step: {avg_train_step_time*1000:.3f} ms - '
            f'avg_val_step: {avg_val_step_time*1000:.3f} ms'
        )

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f'Early stopping en la época {epoch+1}')
                model.load_state_dict(best_state)
                break

    return history


In [33]:
# Dividimos los datos en train/validation

from sklearn.model_selection import train_test_split

# Split de datos (80% train, 20% validation)
indices = np.arange(len(encoder_input_sequences))
train_idx, val_idx = train_test_split(indices, test_size=0.2, random_state=42)

# Datos de entrenamiento
encoder_input_train = encoder_input_sequences[train_idx]
decoder_input_train = decoder_input_sequences[train_idx]
decoder_output_train = decoder_output_sequences[train_idx]

# Datos de validación
encoder_input_val = encoder_input_sequences[val_idx]
decoder_input_val = decoder_input_sequences[val_idx]
decoder_output_val = decoder_output_sequences[val_idx]

print(f"Datos de entrenamiento: {len(train_idx)} muestras")
print(f"Datos de validación: {len(val_idx)} muestras")


Datos de entrenamiento: 95171 muestras
Datos de validación: 23793 muestras


In [ ]:
BATCH_SIZE = 64

train_dataset = TranslationDataset(
    encoder_input=encoder_input_train,
    decoder_input=decoder_input_train,
    decoder_output=decoder_output_train
)

val_dataset = TranslationDataset(
    encoder_input=encoder_input_val,
    decoder_input=decoder_input_val,
    decoder_output=decoder_output_val
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,)
                          #num_workers=4, pin_memory=True, persistent_workers=True, prefetch_factor=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False,)
                        #num_workers=4, pin_memory=True, persistent_workers=True, prefetch_factor=2)

hist = train_model(m_64, train_loader, val_loader, epochs=50, patience=10)
models['n_units_64']['hist'] = hist


Entrenando modelo con 50 épocas y paciencia 10
Device: mps


In [ ]:
BATCH_SIZE = 128

train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=4, pin_memory=True, persistent_workers=True, prefetch_factor=2
    )
val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=4, pin_memory=True, persistent_workers=True, prefetch_factor=2
)

hist = train_model(m_128, train_loader, val_loader, epochs=50, patience=10)
models['n_units_128']['hist'] = hist


In [ ]:
BATCH_SIZE = 256

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

hist = train_model(m_256, train_loader, val_loader, epochs=50, patience=3)
models['n_units_256']['hist'] = hist


#### Comparar modelos


In [ ]:
for n_units in n_units_list:
    hist = models[f'n_units_{n_units}']['hist']
    print(f"Resultados del modelo con {n_units} unidades:")
    # Entrenamiento
    epoch_count = range(1, len(hist['accuracy']) + 1)
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    sns.lineplot(x=epoch_count,  y=hist['loss'], label='train')
    sns.lineplot(x=epoch_count,  y=hist['val_loss'], label='valid')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    sns.lineplot(x=epoch_count,  y=hist['accuracy'], label='train')
    sns.lineplot(x=epoch_count,  y=hist['val_accuracy'], label='valid')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()


Se selecciona el modelo con mejores métricas.


In [ ]:
for name, model_data in models.items():
    history = model_data['hist']
    if history and 'accuracy' in history:
        current_val_loss = history['accuracy'][-1]
        print(f"Model {name}: Final Accuracy = {current_val_loss:.4f}")


In [ ]:
best_model = models['n_units_128']
best_encoder = best_model['encoder']
best_decoder = best_model['decoder']
best_n_units = best_model['n_units']


### 5 - Inferencia


``` python
Step 1:
A deal is a deal -> Encoder -> enc(h1,c1)

enc(h1,c1) + <sos> -> Decoder -> Un + dec(h1,c1)

step 2:
dec(h1,c1) + Un -> Decoder -> trato + dec(h2,c2)

step 3:
dec(h2,c2) + trato -> Decoder -> es + dec(h3,c3)

step 4:
dec(h3,c3) + es -> Decoder -> un + dec(h4,c4)

step 5:
dec(h4,c4) + un -> Decoder -> trato + dec(h5,c5)

step 6:
dec(h5,c5) + trato. -> Decoder -> <eos> + dec(h6,c6)
```


In [ ]:
# Armar los conversores de índice a palabra:
idx2word_input = {v:k for k, v in word2idx_inputs.items()}
idx2word_target = {v:k for k, v in word2idx_outputs.items()}


In [ ]:
def translate_sentence(input_seq):
    best_encoder.eval()
    best_decoder.eval()

    with torch.no_grad():
        # Convertir a tensor y mover al dispositivo
        input_tensor = torch.LongTensor(input_seq).to(device)

        # Se transforma la secuencia de entrada a los estados "h" y "c" de la LSTM
        h, c = best_encoder(input_tensor)

        # Se inicializa la secuencia de entrada al decoder como "<sos>"
        target_seq = torch.LongTensor([[word2idx_outputs['<sos>']]]).to(device)

        # Se obtiene el índice que finaliza la inferencia
        eos = word2idx_outputs['<eos>']

        output_sentence = []
        for _ in range(max_out_len):
            # Predicción del próximo elemento
            output_tokens, h, c = best_decoder(target_seq, h, c)

            # Obtener el índice con mayor probabilidad
            idx = output_tokens[0, 0, :].argmax().item()

            # Si es "end of sentence <eos>" se acaba
            if eos == idx:
                break

            # Transformar idx a palabra
            if idx > 0:
                word = idx2word_target[idx]
                output_sentence.append(word)

            # Actualizar secuencia de entrada con la salida (re-alimentación)
            target_seq = torch.LongTensor([[idx]]).to(device)

        return ' '.join(output_sentence)


In [ ]:
def translate_sentence_sampling(input_seq, temperature=1.0, max_repeat=3):
    best_encoder.eval()
    best_decoder.eval()

    with torch.no_grad():
        input_tensor = torch.LongTensor(input_seq).to(device)
        h, c = best_encoder(input_tensor)

        target_seq = torch.LongTensor([[word2idx_outputs['<sos>']]]).to(device)
        eos = word2idx_outputs['<eos>']

        output_sentence = []
        last_idx = None
        repeat_count = 0

        for _ in range(max_out_len):
            output_tokens, h, c = best_decoder(target_seq, h, c)

            logits = output_tokens[0, 0, :] / temperature
            logits = logits - logits.max()  # estabilidad numérica
            probs = torch.softmax(logits, dim=0)

            idx = torch.multinomial(probs, 1).item()

            if eos == idx:
                break

            # Si se repite el mismo token muchas veces, forzar corte
            if idx == last_idx:
                repeat_count += 1
                if repeat_count >= max_repeat:
                    break
            else:
                repeat_count = 0
            last_idx = idx

            # Si el token es <sos> o <pad>, ignorar y no agregar a la secuencia
            if idx > 0 and idx != word2idx_outputs.get('<sos>', -1):
                word = idx2word_target.get(idx, '')
                output_sentence.append(word)

            target_seq = torch.LongTensor([[idx]]).to(device)

        return ' '.join(output_sentence[:max_out_len])


In [ ]:
def translate_sentence_beam_search_stochastic(input_seq, beam_width=3, temperature=1.0):
    best_encoder.eval()
    best_decoder.eval()

    with torch.no_grad():
        input_tensor = torch.LongTensor(input_seq).to(device)
        h, c = best_encoder(input_tensor)

        sequences = [(['<sos>'], 0.0, (h, c))]  # (sequence, score, states)

        for _ in range(max_out_len):
            all_candidates = []
            for seq, score, states in sequences:
                h_state, c_state = states

                target_seq = torch.LongTensor([[word2idx_outputs.get(seq[-1], 0)]]).to(device)

                output_tokens, h_new, c_new = best_decoder(target_seq, h_state, c_state)

                logits = output_tokens[0, 0, :] / temperature
                logits = logits - logits.max()  # estabilidad numérica
                probs = torch.softmax(logits, dim=0)

                # Obtener las top beam_width predicciones
                top_probs, top_indices = torch.topk(probs, beam_width)

                for i in range(beam_width):
                    idx = top_indices[i].item()
                    word = idx2word_target.get(idx, '')
                    candidate = (seq + [word], score - torch.log(top_probs[i]).item(), (h_new, c_new))
                    all_candidates.append(candidate)

            # Ordenar todas las secuencias candidatas por score y quedarse con las mejores beam_width
            ordered = sorted(all_candidates, key=lambda tup: tup[1])
            sequences = ordered[:beam_width]

        # Seleccionar la secuencia con el mejor score que no termine en <eos>
        best_sequence = sequences[0][0]
        if '<eos>' in best_sequence:
            best_sequence = best_sequence[:best_sequence.index('<eos>')]

        return ' '.join(best_sequence[1:])


In [ ]:
i = np.random.choice(len(input_sentences))
input_seq = encoder_input_sequences[i:i+1]
translation = translate_sentence(input_seq)
print('-')
print('Input:', input_sentences[i])
print('Response:', translation)


In [ ]:
# Ejemplo de uso con muestreo aleatorio
i = np.random.choice(len(input_sentences))
input_seq = encoder_input_sequences[i:i+1]
#translation = translate_sentence_sampling(input_seq, temperature=0.2)
translation = translate_sentence_beam_search_stochastic(input_seq, temperature=0.8)
print('-')
print('Input:', input_sentences[i])
print('Response (sampling):', translation)


In [ ]:
def generate_translations(input_test="", temperature=1.0):
    if input_test == "":
        # Seleccionar una oración al azar del dataset
        i = np.random.choice(len(input_sentences))
        encoder_sequence_test = encoder_input_sequences[i:i+1]
        print('Input:', input_sentences[i])
    else:
        # Procesar la oración de entrada
        integer_seq_test = input_tokenizer.texts_to_sequences([input_test])[0]
        print("Representacion en vector de tokens de ids", integer_seq_test)
        encoder_sequence_test = pad_sequences([integer_seq_test], maxlen=max_input_len)
        print("Padding del vector:", encoder_sequence_test)

        print('Input:', input_test)

    translation = translate_sentence(encoder_sequence_test)
    translation_2 = translate_sentence_beam_search_stochastic(encoder_sequence_test, temperature=temperature)

    print('Response:', translation)
    print('Response (Beam Search):', translation_2)


In [ ]:
input_test = "My mother say hi."
generate_translations(input_test=input_test, temperature=0.1)


In [ ]:
input_test = "Every end is a new beginning"
generate_translations(input_test=input_test, temperature=0.1)


In [ ]:
input_test = "The best of both worlds"
generate_translations(input_test=input_test, temperature=1.3)


In [ ]:
input_test = "I know what you mean"
generate_translations(input_test=input_test, temperature=0.1)


In [ ]:
input_test = "Give me a break"
generate_translations(input_test=input_test, temperature=0.1)


In [ ]:
input_test = "where there is a will there is a way"
generate_translations(input_test=input_test, temperature=0.1)


### 6 - Conclusión

A primera vista parece que el modelo tendría que funcionar muy bien por el accuracy alcanzado. La realidad es que las respuestas no tienen que ver demasiado con la pregunta/traducción pero la respuesta en si tiene bastante coherencia.

**Solución implementada - Dataset de PyTorch:**

Se implementó un `Dataset` basado en `torch.utils.data.Dataset` que:
- Genera los batches on-the-fly durante el entrenamiento
- PyTorch CrossEntropyLoss trabaja directamente con índices (sin necesidad de one-hot)
- Permite entrenar con datasets mucho más grandes sin agotar la RAM
- Aumentamos de 10,000 a 118,964 muestras de entrenamiento

Otras mejoras posibles:
- Transfer learning evitando tener que entrenar todo el modelo desde cero
- Utilizar embeddings pre-entrenados para español también
- Aumentar aún más el dataset aprovechando el DataLoader
